# Find Valid Stolen Base Attempts and if they are Successful or Not

Loading all of the game events csv files /intial processing

In [ ]:
library(tidyverse)

In [ ]:
file_list <- list.files(pattern = "^game_events.*\\.csv$")
print(file_list)

 [1] "game_events-1900_01_TeamKJ_TeamB.csv" 
 [2] "game_events-1900_02_TeamKJ_TeamB.csv" 
 [3] "game_events-1900_03_TeamKJ_TeamB.csv" 
 [4] "game_events-1900_04_TeamKK_TeamB.csv" 
 [5] "game_events-1900_05_TeamKK_TeamB.csv" 
 [6] "game_events-1900_06_TeamKL_TeamB.csv" 
 [7] "game_events-1900_07_TeamKL_TeamB.csv" 
 [8] "game_events-1900_08_TeamKL_TeamB.csv" 
 [9] "game_events-1900_09_TeamKK_TeamB.csv" 
[10] "game_events-1901_01_TeamLG_TeamA3.csv"
[11] "game_events-1901_02_TeamLG_TeamA3.csv"
[12] "game_events-1901_03_TeamLG_TeamA3.csv"
[13] "game_events-1901_04_TeamLI_TeamA3.csv"
[14] "game_events-1901_05_TeamLI_TeamA3.csv"
[15] "game_events-1901_06_TeamLI_TeamA3.csv"
[16] "game_events-1901_07_TeamLK_TeamB.csv" 
[17] "game_events-1901_08_TeamLK_TeamB.csv" 
[18] "game_events-1901_09_TeamLK_TeamB.csv" 
[19] "game_events-1901_10_TeamLJ_TeamB.csv" 
[20] "game_events-1901_11_TeamLJ_TeamB.csv" 
[21] "game_events-1901_12_TeamLJ_TeamB.csv" 
[22] "game_events-1901_13_TeamLL_TeamB.csv" 
[23] "game

In [ ]:
combined_event_data <- data.frame()

In [ ]:
game_id <- 1
for (file in file_list) {
  event_data <- read_csv(paste0("/content/", file))
  event_data <- event_data %>%
    mutate(game_id = game_id) # Add game_id column with unique value
  combined_event_data <- bind_rows(combined_event_data, event_data)
  game_id <- game_id + 1 # Increment game_id counter
}

New names:
• `` -> `...1`
Rows: 939 Columns: 8
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): game_str
dbl (6): ...1, play_id, play_per_game, timestamp, player_position, event_code
lgl (1): at_bat

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
New names:
• `` -> `...1`
Rows: 964 Columns: 8
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): game_str
dbl (6): ...1, play_id, play_per_game, timestamp, player_position, event_code
lgl (1): at_bat

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
New names:
• `` -> `...1`
Rows: 742 Columns: 8
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): game_str
dbl (6): ...1, play

Looking at the situations where a catcher threw the ball to the 2nd baseman, these should indicate where a stolen base attempt was made

In [ ]:
stolen_base_main = combined_event_data %>% select(game_str, game_id, play_id, player_position, event_code, play_per_game) %>% #selecting only the columns needed
  group_by(game_id, play_per_game) %>% #each scenario will be a different play
    filter(n() > 3) %>% #n() > 3 means the standard pitch, miss, catch (or nothing eventful happens) is eliminated
      filter((player_position == 2 & event_code == 3) |
              (player_position == 4 & event_code == 2)) %>%
        filter(n() > 1) %>% print #since many plays have only (4,2) or only (2,3) but not both

# A tibble: 132 × 6
# Groups:   game_id, play_per_game [66]
   game_str             game_id play_id player_position event_code play_per_game
   <chr>                  <dbl>   <dbl>           <dbl>      <dbl>         <dbl>
 1 1900_05_TeamKK_TeamB       5      56               2          3            56
 2 1900_05_TeamKK_TeamB       5      56               4          2            56
 3 1900_05_TeamKK_TeamB       5     218               2          3           219
 4 1900_05_TeamKK_TeamB       5     218               4          2           219
 5 1900_08_TeamKL_TeamB       8     270               2          3           270
 6 1900_08_TeamKL_TeamB       8     270               4          2           270
 7 1901_02_TeamLG_Team…      11     227               2          3           227
 8 1901_02_TeamLG_Team…      11     227               4          2           227
 9 1901_02_TeamLG_Team…      11     282               2          3           282
10 1901_02_TeamLG_Team…      11     282          

Checking what happens with one play, and connecting that one play with information from game info (shows us who is on what base) and later player position

In [ ]:
#Now lets see what happens with one play

#isolate one play
one_play = combined_event_data %>%
  filter(game_id == 5, play_per_game == 56) %>%
    filter((player_position == 2 & event_code == 3) |
           (player_position == 4 & event_code == 2))
one_play

...1,game_str,play_id,at_bat,play_per_game,timestamp,player_position,event_code,game_id
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
194,1900_05_TeamKK_TeamB,56,NA,56,2231896,2,3,5
195,1900_05_TeamKK_TeamB,56,NA,56,2233184,4,2,5


In [ ]:
#Combine player_pos table with corresponding game_id and play
game_5_team_b = read_csv("/content/game_info-1900_05_TeamKK_TeamB.csv")

New names:
• `` -> `...1`
Rows: 255 Columns: 21
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (4): game_str, home_team, away_team, top_bottom_inning
dbl (16): ...1, play_per_game, inning, pitcher, catcher, first_base, second_...
lgl  (1): at_bat

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [ ]:
one_play_corr = game_5_team_b
one_play_corr %>% filter(between(play_per_game, 55, 57))

...1,game_str,home_team,away_team,at_bat,play_per_game,inning,top_bottom_inning,pitcher,catcher,⋯,second_base,third_base,shortstop,left_field,center_field,right_field,batter,first_baserunner,second_baserunner,third_baserunner
<dbl>,<chr>,<chr>,<chr>,<lgl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
45,1900_05_TeamKK_TeamB,TeamB,TeamKK,NA,55,2,Bottom,7119,3304,⋯,4140,9752,6861,6279,6608,6188,2488,0,0,0
46,1900_05_TeamKK_TeamB,TeamB,TeamKK,NA,56,2,Bottom,7119,3304,⋯,4140,9752,6861,6279,6608,6188,2488,0,0,0
47,1900_05_TeamKK_TeamB,TeamB,TeamKK,NA,57,2,Bottom,7119,3304,⋯,4140,9752,6861,6279,6608,6188,2488,0,0,0


In [ ]:
player_position = read_csv("/content/player_pos-1900_05_TeamKK_TeamB.csv")
# checked player position

New names:
• `` -> `...1`
Rows: 177178 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): game_str
dbl (6): ...1, play_id, timestamp, player_position, field_x, field_y

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [ ]:
one_play_player = player_position
one_play_player %>% filter(play_id == 56 & player_position == 11) #same game, same play_id, first base runner tracking

...1,game_str,play_id,timestamp,player_position,field_x,field_y
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
37207,1900_05_TeamKK_TeamB,56,2230576,11,48.5067,78.4392
37208,1900_05_TeamKK_TeamB,56,2230609,11,48.0558,78.9045
37209,1900_05_TeamKK_TeamB,56,2230642,11,47.6043,79.3701
37210,1900_05_TeamKK_TeamB,56,2230675,11,47.1525,79.8366
37211,1900_05_TeamKK_TeamB,56,2230708,11,46.6995,80.3046
37212,1900_05_TeamKK_TeamB,56,2230741,11,46.2453,80.7741
37213,1900_05_TeamKK_TeamB,56,2230774,11,45.7896,81.2463
37214,1900_05_TeamKK_TeamB,56,2230807,11,45.3324,81.7212
37215,1900_05_TeamKK_TeamB,56,2230840,11,44.8731,82.1997


Conclusion from one play: it seems like there is an error in the game info data because according to player position there is a runner who stole 2nd base. This is good to keep in mind when checking the rest of the cases!
Now: check the rest of the cases with game info data (still only looking at catcher throwing to 2nd baseman)

In [ ]:
#continue checking game info for rest of situations
file_list <- list.files(pattern = "^game_info.*\\.csv$")
combined_game_info_data <- data.frame()
for (file in file_list) {
  game_info_data <- read_csv(paste0("/content/", file))
  combined_game_info_data <- bind_rows(combined_game_info_data, game_info_data)
}

New names:
• `` -> `...1`
Rows: 253 Columns: 21
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (4): game_str, home_team, away_team, top_bottom_inning
dbl (16): ...1, play_per_game, inning, pitcher, catcher, first_base, second_...
lgl  (1): at_bat

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
New names:
• `` -> `...1`
Rows: 251 Columns: 21
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (4): game_str, home_team, away_team, top_bottom_inning
dbl (16): ...1, play_per_game, inning, pitcher, catcher, first_base, second_...
lgl  (1): at_bat

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
New names:
• `` -> `...1`
Rows: 172 Columns: 21
── Column specification ────────────────

In [ ]:
combined_game_info_data

...1,game_str,home_team,away_team,at_bat,play_per_game,inning,top_bottom_inning,pitcher,catcher,⋯,second_base,third_base,shortstop,left_field,center_field,right_field,batter,first_baserunner,second_baserunner,third_baserunner
<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1900_01_TeamKJ_TeamB,TeamB,TeamKJ,NA,2,1,Top,1571,2456,⋯,1201,2653,2614,2488,2061,2279,8180,0,0,0
2,1900_01_TeamKJ_TeamB,TeamB,TeamKJ,NA,3,1,Top,1571,2456,⋯,1201,2653,2614,2488,2061,2279,8180,0,0,0
3,1900_01_TeamKJ_TeamB,TeamB,TeamKJ,NA,4,1,Top,1571,2456,⋯,1201,2653,2614,2488,2061,2279,8180,0,0,0
4,1900_01_TeamKJ_TeamB,TeamB,TeamKJ,NA,5,1,Top,1571,2456,⋯,1201,2653,2614,2488,2061,2279,6752,0,0,0
5,1900_01_TeamKJ_TeamB,TeamB,TeamKJ,NA,6,1,Top,1571,2456,⋯,1201,2653,2614,2488,2061,2279,6752,0,0,0
6,1900_01_TeamKJ_TeamB,TeamB,TeamKJ,NA,7,1,Top,1571,2456,⋯,1201,2653,2614,2488,2061,2279,6752,0,0,0
7,1900_01_TeamKJ_TeamB,TeamB,TeamKJ,NA,8,1,Top,1571,2456,⋯,1201,2653,2614,2488,2061,2279,7225,6752,0,0
8,1900_01_TeamKJ_TeamB,TeamB,TeamKJ,NA,10,1,Top,1571,2456,⋯,1201,2653,2614,2488,2061,2279,7225,6752,0,0
9,1900_01_TeamKJ_TeamB,TeamB,TeamKJ,NA,11,1,Top,1571,2456,⋯,1201,2653,2614,2488,2061,2279,7225,6752,0,0


Verifying that the data we are now handling matches the one play we checked earlier

In [ ]:
combined_game_info_data %>% filter(between(play_per_game, 55, 57) & game_str == "1900_05_TeamKK_TeamB")

...1,game_str,home_team,away_team,at_bat,play_per_game,inning,top_bottom_inning,pitcher,catcher,⋯,second_base,third_base,shortstop,left_field,center_field,right_field,batter,first_baserunner,second_baserunner,third_baserunner
<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
45,1900_05_TeamKK_TeamB,TeamB,TeamKK,NA,55,2,Bottom,7119,3304,⋯,4140,9752,6861,6279,6608,6188,2488,0,0,0
46,1900_05_TeamKK_TeamB,TeamB,TeamKK,NA,56,2,Bottom,7119,3304,⋯,4140,9752,6861,6279,6608,6188,2488,0,0,0
47,1900_05_TeamKK_TeamB,TeamB,TeamKK,NA,57,2,Bottom,7119,3304,⋯,4140,9752,6861,6279,6608,6188,2488,0,0,0


Here we are making sure that each stolen base attempt is represented once.

After, we connect each stolen base attempt with information from the game info table. Using the play of and one play after, so we know what happened.

We can then determine whether each stolen base attempt was succcessful or not.

In [ ]:
full = stolen_base_main %>%
  distinct(game_str, play_per_game) %>%
    select(game_str, play_per_game)
full

Adding missing grouping variables: `game_id`


game_id,game_str,play_per_game
<dbl>,<chr>,<dbl>
5,1900_05_TeamKK_TeamB,56
5,1900_05_TeamKK_TeamB,219
8,1900_08_TeamKL_TeamB,270
11,1901_02_TeamLG_TeamA3,227
11,1901_02_TeamLG_TeamA3,282
12,1901_03_TeamLG_TeamA3,187
14,1901_05_TeamLI_TeamA3,12
16,1901_07_TeamLK_TeamB,124
18,1901_09_TeamLK_TeamB,65


In [ ]:
results_full <- data.frame()
counter <- 0
for (i in 1:nrow(full)) {
  row <- full[i,]
  res <- combined_game_info_data %>% filter(
    between(play_per_game, as.numeric(row[["play_per_game"]]), as.numeric(row[["play_per_game"]])+1) &
      game_str == row[["game_str"]]
  ) %>%
    arrange(play_per_game) %>%
    slice(which(abs(play_per_game - as.numeric(row[["play_per_game"]])) <= 1)) %>%
    mutate(stolen_event_id = counter)
  counter <- counter + 1

  results_full <- bind_rows(results_full, res)
}

results_full$type <- vector("character", nrow(results_full))



results_full

# why are there 122 rows when there should be 66 (attempts) * 2 (rows per attempt) = 132 rows, missing 10 rows

...1,game_str,home_team,away_team,at_bat,play_per_game,inning,top_bottom_inning,pitcher,catcher,⋯,shortstop,left_field,center_field,right_field,batter,first_baserunner,second_baserunner,third_baserunner,stolen_event_id,type
<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
46,1900_05_TeamKK_TeamB,TeamB,TeamKK,NA,56,2,Bottom,7119,3304,⋯,6861,6279,6608,6188,2488,0,0,0,0,
47,1900_05_TeamKK_TeamB,TeamB,TeamKK,NA,57,2,Bottom,7119,3304,⋯,6861,6279,6608,6188,2488,0,0,0,0,
204,1900_05_TeamKK_TeamB,TeamB,TeamKK,NA,219,7,Top,1516,1041,⋯,2614,2488,2061,2279,6188,6861,4140,0,1,
205,1900_05_TeamKK_TeamB,TeamB,TeamKK,NA,220,7,Top,1516,1041,⋯,2614,2488,2061,2279,6188,6861,4140,0,1,
255,1900_08_TeamKL_TeamB,TeamB,TeamKL,NA,270,9,Top,2995,1041,⋯,2614,2488,2889,2279,6092,0,3833,0,2,
256,1900_08_TeamKL_TeamB,TeamB,TeamKL,NA,271,9,Top,2995,1041,⋯,2614,2488,2889,2279,6092,0,3833,0,2,
170,1901_02_TeamLG_TeamA3,TeamA3,TeamLG,NA,227,6,Bottom,3986,6025,⋯,6685,9871,7927,5626,1217,0,0,0,3,
171,1901_02_TeamLG_TeamA3,TeamA3,TeamLG,NA,228,6,Bottom,3986,6025,⋯,6685,9871,7927,5626,1217,0,0,0,3,
214,1901_02_TeamLG_TeamA3,TeamA3,TeamLG,NA,282,7,Bottom,3986,6025,⋯,6685,9871,7927,5626,2911,0,1470,0,4,


In [ ]:
#try to group situations
# identify situations where its 0, 0, 0 everywhere, or where the player moves while the batter remains, or player
# "dissapears" while batter remains
results_full$throw <- vector("character", nrow(results_full))


for (i in 1:(nrow(results_full) - 1)) {
   results_full[i,"throw"] = "2b"
  # all zeros
  if (results_full[i,"first_baserunner"] == 0 & results_full[i,"second_baserunner"] == 0 &
    results_full[i,"third_baserunner"] == 0 & results_full[i+1,"first_baserunner"] == 0 &
      results_full[i+1,"second_baserunner"] == 0 & results_full[i+1,"third_baserunner"] == 0 &
      results_full[i, "stolen_event_id"] == results_full[i+1, "stolen_event_id"]) {
    results_full[i, "type"] = "nothing"
    results_full[i+1, "type"] = "nothing"
  }

  # stolen
  else if (results_full[i,"first_baserunner"] == results_full[i+1,"second_baserunner"] &
           results_full[i,"batter"] == results_full[i+1,"batter"] &
      results_full[i, "stolen_event_id"] == results_full[i+1, "stolen_event_id"]) {
             results_full[i, "type"] = "stolen"
             results_full[i+1, "type"] = "stolen"
  }

  #caught stealing
  else if (results_full[i,"first_baserunner"] != 0 &
           results_full[i+1,"first_baserunner"] == 0 &
           results_full[i+1,"second_baserunner"] == 0 &
           results_full[i,"batter"] == results_full[i+1,"batter"] &
      results_full[i, "stolen_event_id"] == results_full[i+1, "stolen_event_id"]) {
            results_full[i, "type"] = "caught"
            results_full[i+1, "type"] = "caught"

  }

  else if (results_full[i, "stolen_event_id"] == results_full[i+1, "stolen_event_id"]) {
            results_full[i, "type"] = "other"
            results_full[i+1, "type"] = "other"
  }
  else if (results_full[i, "stolen_event_id"] != results_full[i+1, "stolen_event_id"] & results_full[i, "type"] == "") {
      results_full[i, "type"] = "alone"
  }
}

results_full


...1,game_str,home_team,away_team,at_bat,play_per_game,inning,top_bottom_inning,pitcher,catcher,⋯,left_field,center_field,right_field,batter,first_baserunner,second_baserunner,third_baserunner,stolen_event_id,type,throw
<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
46,1900_05_TeamKK_TeamB,TeamB,TeamKK,NA,56,2,Bottom,7119,3304,⋯,6279,6608,6188,2488,0,0,0,0,nothing,2b
47,1900_05_TeamKK_TeamB,TeamB,TeamKK,NA,57,2,Bottom,7119,3304,⋯,6279,6608,6188,2488,0,0,0,0,nothing,2b
204,1900_05_TeamKK_TeamB,TeamB,TeamKK,NA,219,7,Top,1516,1041,⋯,2488,2061,2279,6188,6861,4140,0,1,other,2b
205,1900_05_TeamKK_TeamB,TeamB,TeamKK,NA,220,7,Top,1516,1041,⋯,2488,2061,2279,6188,6861,4140,0,1,other,2b
255,1900_08_TeamKL_TeamB,TeamB,TeamKL,NA,270,9,Top,2995,1041,⋯,2488,2889,2279,6092,0,3833,0,2,other,2b
256,1900_08_TeamKL_TeamB,TeamB,TeamKL,NA,271,9,Top,2995,1041,⋯,2488,2889,2279,6092,0,3833,0,2,other,2b
170,1901_02_TeamLG_TeamA3,TeamA3,TeamLG,NA,227,6,Bottom,3986,6025,⋯,9871,7927,5626,1217,0,0,0,3,nothing,2b
171,1901_02_TeamLG_TeamA3,TeamA3,TeamLG,NA,228,6,Bottom,3986,6025,⋯,9871,7927,5626,1217,0,0,0,3,nothing,2b
214,1901_02_TeamLG_TeamA3,TeamA3,TeamLG,NA,282,7,Bottom,3986,6025,⋯,9871,7927,5626,2911,0,1470,0,4,other,2b


Conclusion: there are some missing rows of data (alone rows), there are some clean stolen/caught stealing situtations. There are also a ton of "other" or "missing" situtations that need to be looked into.

Now: use the situtation where the catcher throws to the shortstop and follow the same steps as above to filter and sort.

In [ ]:
stolen_base_other = combined_event_data %>% select(game_str, game_id, play_id, player_position, event_code, play_per_game) %>% #selecting only the columns needed
  group_by(game_id, play_per_game) %>% #each scenario will be a different play
    filter(n() > 3) %>% #n() > 3 means the standard pitch, miss, catch (or nothing eventful happens) is eliminated
      filter((player_position == 2 & event_code == 3) |
              (player_position == 6 & event_code == 2)) %>%
        filter(n() > 1) %>% print #since many plays have only (4,2) or only (2,3) but not both

# A tibble: 141 × 6
# Groups:   game_id, play_per_game [70]
   game_str             game_id play_id player_position event_code play_per_game
   <chr>                  <dbl>   <dbl>           <dbl>      <dbl>         <dbl>
 1 1901_02_TeamLG_Team…      11      99               2          3            99
 2 1901_02_TeamLG_Team…      11      99               6          2            99
 3 1901_05_TeamLI_Team…      14      12               2          3            12
 4 1901_05_TeamLI_Team…      14      12               6          2            12
 5 1901_05_TeamLI_Team…      14      81               2          3            81
 6 1901_05_TeamLI_Team…      14      81               6          2            81
 7 1901_09_TeamLK_TeamB      18     145               2          3           146
 8 1901_09_TeamLK_TeamB      18     145               6          2           146
 9 1901_10_TeamLJ_TeamB      19      81               2          3            84
10 1901_10_TeamLJ_TeamB      19      81          

In [ ]:
stolen_base_other

game_str,game_id,play_id,player_position,event_code,play_per_game
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1901_02_TeamLG_TeamA3,11,99,2,3,99
1901_02_TeamLG_TeamA3,11,99,6,2,99
1901_05_TeamLI_TeamA3,14,12,2,3,12
1901_05_TeamLI_TeamA3,14,12,6,2,12
1901_05_TeamLI_TeamA3,14,81,2,3,81
1901_05_TeamLI_TeamA3,14,81,6,2,81
1901_09_TeamLK_TeamB,18,145,2,3,146
1901_09_TeamLK_TeamB,18,145,6,2,146
1901_10_TeamLJ_TeamB,19,81,2,3,84


In [ ]:
other_full = stolen_base_other %>%
  distinct(game_str, play_per_game) %>%
    select(game_str, play_per_game)
other_full

Adding missing grouping variables: `game_id`


game_id,game_str,play_per_game
<dbl>,<chr>,<dbl>
11,1901_02_TeamLG_TeamA3,99
14,1901_05_TeamLI_TeamA3,12
14,1901_05_TeamLI_TeamA3,81
18,1901_09_TeamLK_TeamB,146
19,1901_10_TeamLJ_TeamB,84
19,1901_10_TeamLJ_TeamB,211
20,1901_11_TeamLJ_TeamB,39
20,1901_11_TeamLJ_TeamB,119
20,1901_11_TeamLJ_TeamB,152


In [ ]:
other_results_full <- data.frame()
counter <- 0
for (i in 1:nrow(other_full)) {
  row <- other_full[i,]
  res <- combined_game_info_data %>% filter(
    between(play_per_game, as.numeric(row[["play_per_game"]]), as.numeric(row[["play_per_game"]])+1) &
      game_str == row[["game_str"]]
  ) %>%
    arrange(play_per_game) %>%
    slice(which(abs(play_per_game - as.numeric(row[["play_per_game"]])) <= 1)) %>%
    mutate(stolen_event_id = counter)
  counter <- counter + 1

  other_results_full <- bind_rows(other_results_full, res)
}

other_results_full$type <- vector("character", nrow(other_results_full))



other_results_full

#there should be 140 rows but there are only 129

...1,game_str,home_team,away_team,at_bat,play_per_game,inning,top_bottom_inning,pitcher,catcher,⋯,shortstop,left_field,center_field,right_field,batter,first_baserunner,second_baserunner,third_baserunner,stolen_event_id,type
<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
70,1901_02_TeamLG_TeamA3,TeamA3,TeamLG,NA,99,2,Bottom,8559,6025,⋯,6685,9871,7927,5626,1693,1959,0,2320,0,
71,1901_02_TeamLG_TeamA3,TeamA3,TeamLG,NA,100,2,Bottom,8559,6025,⋯,6685,9871,7927,5626,1693,1959,0,2320,0,
12,1901_05_TeamLI_TeamA3,TeamA3,TeamLI,NA,12,1,Top,1896,2702,⋯,1959,1721,2320,2946,9389,9460,0,0,1,
13,1901_05_TeamLI_TeamA3,TeamA3,TeamLI,NA,13,1,Top,1896,2702,⋯,1959,1721,2320,2946,9907,0,0,9460,1,
76,1901_05_TeamLI_TeamA3,TeamA3,TeamLI,NA,81,2,Top,1896,2702,⋯,1959,1721,2320,2946,9460,7385,0,0,2,
77,1901_05_TeamLI_TeamA3,TeamA3,TeamLI,NA,82,2,Top,1896,2702,⋯,1959,1721,2320,2946,9389,9460,0,0,2,
136,1901_09_TeamLK_TeamB,TeamB,TeamLK,NA,146,5,Bottom,6922,6607,⋯,7830,7908,8568,5933,1748,0,0,0,3,
137,1901_09_TeamLK_TeamB,TeamB,TeamLK,NA,147,5,Bottom,6922,6607,⋯,7830,7908,8568,5933,1748,0,0,0,3,
72,1901_10_TeamLJ_TeamB,TeamB,TeamLJ,NA,84,3,Top,1155,2908,⋯,1748,1659,1450,1657,6327,0,0,0,4,


In [ ]:
#try to group situations
# identify situations where its 0, 0, 0 everywhere, or where the player moves while the batter remains, or player
# "dissapears" while batter remains

other_results_full$throw <- vector("character", nrow(other_results_full))


for (i in 1:(nrow(other_results_full) - 1)) {
  other_results_full[i, "throw"] = "ss"
  # all zeros
  if (other_results_full[i,"first_baserunner"] == 0 & other_results_full[i,"second_baserunner"] == 0 &
    other_results_full[i,"third_baserunner"] == 0 & other_results_full[i+1,"first_baserunner"] == 0 &
      other_results_full[i+1,"second_baserunner"] == 0 & other_results_full[i+1,"third_baserunner"] == 0 &
      other_results_full[i, "stolen_event_id"] == other_results_full[i+1, "stolen_event_id"]) {
    other_results_full[i, "type"] = "nothing"
    other_results_full[i+1, "type"] = "nothing"
  }

  # stolen
  else if (other_results_full[i,"first_baserunner"] == other_results_full[i+1,"second_baserunner"] &
           other_results_full[i,"batter"] == other_results_full[i+1,"batter"] &
      other_results_full[i, "stolen_event_id"] == other_results_full[i+1, "stolen_event_id"]) {
             other_results_full[i, "type"] = "stolen"
             other_results_full[i+1, "type"] = "stolen"
  }

  #caught stealing
  else if (other_results_full[i,"first_baserunner"] != 0 &
           other_results_full[i+1,"first_baserunner"] == 0 &
           other_results_full[i+1,"second_baserunner"] == 0 &
           other_results_full[i,"batter"] == other_results_full[i+1,"batter"] &
      other_results_full[i, "stolen_event_id"] == other_results_full[i+1, "stolen_event_id"]) {
            other_results_full[i, "type"] = "caught"
            other_results_full[i+1, "type"] = "caught"

  }

  else if (other_results_full[i, "stolen_event_id"] == other_results_full[i+1, "stolen_event_id"]) {
            other_results_full[i, "type"] = "other"
            other_results_full[i+1, "type"] = "other"
  }

   else if (other_results_full[i, "stolen_event_id"] != other_results_full[i+1, "stolen_event_id"]
    & other_results_full[i, "type"] == "") {
      other_results_full[i, "type"] = "alone"
  }
}

other_results_full[nrow(other_results_full), "throw"] = "ss"
other_results_full


...1,game_str,home_team,away_team,at_bat,play_per_game,inning,top_bottom_inning,pitcher,catcher,⋯,left_field,center_field,right_field,batter,first_baserunner,second_baserunner,third_baserunner,stolen_event_id,type,throw
<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
70,1901_02_TeamLG_TeamA3,TeamA3,TeamLG,NA,99,2,Bottom,8559,6025,⋯,9871,7927,5626,1693,1959,0,2320,0,other,ss
71,1901_02_TeamLG_TeamA3,TeamA3,TeamLG,NA,100,2,Bottom,8559,6025,⋯,9871,7927,5626,1693,1959,0,2320,0,other,ss
12,1901_05_TeamLI_TeamA3,TeamA3,TeamLI,NA,12,1,Top,1896,2702,⋯,1721,2320,2946,9389,9460,0,0,1,other,ss
13,1901_05_TeamLI_TeamA3,TeamA3,TeamLI,NA,13,1,Top,1896,2702,⋯,1721,2320,2946,9907,0,0,9460,1,other,ss
76,1901_05_TeamLI_TeamA3,TeamA3,TeamLI,NA,81,2,Top,1896,2702,⋯,1721,2320,2946,9460,7385,0,0,2,other,ss
77,1901_05_TeamLI_TeamA3,TeamA3,TeamLI,NA,82,2,Top,1896,2702,⋯,1721,2320,2946,9389,9460,0,0,2,other,ss
136,1901_09_TeamLK_TeamB,TeamB,TeamLK,NA,146,5,Bottom,6922,6607,⋯,7908,8568,5933,1748,0,0,0,3,nothing,ss
137,1901_09_TeamLK_TeamB,TeamB,TeamLK,NA,147,5,Bottom,6922,6607,⋯,7908,8568,5933,1748,0,0,0,3,nothing,ss
72,1901_10_TeamLJ_TeamB,TeamB,TeamLJ,NA,84,3,Top,1155,2908,⋯,1659,1450,1657,6327,0,0,0,4,nothing,ss


combining the two types of situations (where catcher throws to 2nd baseman vs shortstop) for ease.
After, getting rid of columns that we don't need to simplify.

In [ ]:
stolen_base_attempts <- rbind(results_full, other_results_full)
stolen_base_attempts

...1,game_str,home_team,away_team,at_bat,play_per_game,inning,top_bottom_inning,pitcher,catcher,⋯,left_field,center_field,right_field,batter,first_baserunner,second_baserunner,third_baserunner,stolen_event_id,type,throw
<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
46,1900_05_TeamKK_TeamB,TeamB,TeamKK,NA,56,2,Bottom,7119,3304,⋯,6279,6608,6188,2488,0,0,0,0,nothing,2b
47,1900_05_TeamKK_TeamB,TeamB,TeamKK,NA,57,2,Bottom,7119,3304,⋯,6279,6608,6188,2488,0,0,0,0,nothing,2b
204,1900_05_TeamKK_TeamB,TeamB,TeamKK,NA,219,7,Top,1516,1041,⋯,2488,2061,2279,6188,6861,4140,0,1,other,2b
205,1900_05_TeamKK_TeamB,TeamB,TeamKK,NA,220,7,Top,1516,1041,⋯,2488,2061,2279,6188,6861,4140,0,1,other,2b
255,1900_08_TeamKL_TeamB,TeamB,TeamKL,NA,270,9,Top,2995,1041,⋯,2488,2889,2279,6092,0,3833,0,2,other,2b
256,1900_08_TeamKL_TeamB,TeamB,TeamKL,NA,271,9,Top,2995,1041,⋯,2488,2889,2279,6092,0,3833,0,2,other,2b
170,1901_02_TeamLG_TeamA3,TeamA3,TeamLG,NA,227,6,Bottom,3986,6025,⋯,9871,7927,5626,1217,0,0,0,3,nothing,2b
171,1901_02_TeamLG_TeamA3,TeamA3,TeamLG,NA,228,6,Bottom,3986,6025,⋯,9871,7927,5626,1217,0,0,0,3,nothing,2b
214,1901_02_TeamLG_TeamA3,TeamA3,TeamLG,NA,282,7,Bottom,3986,6025,⋯,9871,7927,5626,2911,0,1470,0,4,other,2b


In [ ]:
df = subset(stolen_base_attempts, select = -c(first_base, second_base, third_base, shortstop, left_field, center_field, right_field) )
df


,...1,game_str,home_team,away_team,at_bat,play_per_game,inning,top_bottom_inning,pitcher,catcher,batter,first_baserunner,second_baserunner,third_baserunner,stolen_event_id,type,throw
,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
1,46,1900_05_TeamKK_TeamB,TeamB,TeamKK,NA,56,2,Bottom,7119,3304,2488,0,0,0,0,nothing,2b
2,47,1900_05_TeamKK_TeamB,TeamB,TeamKK,NA,57,2,Bottom,7119,3304,2488,0,0,0,0,nothing,2b
3,204,1900_05_TeamKK_TeamB,TeamB,TeamKK,NA,219,7,Top,1516,1041,6188,6861,4140,0,1,other,2b
4,205,1900_05_TeamKK_TeamB,TeamB,TeamKK,NA,220,7,Top,1516,1041,6188,6861,4140,0,1,other,2b
5,255,1900_08_TeamKL_TeamB,TeamB,TeamKL,NA,270,9,Top,2995,1041,6092,0,3833,0,2,other,2b
6,256,1900_08_TeamKL_TeamB,TeamB,TeamKL,NA,271,9,Top,2995,1041,6092,0,3833,0,2,other,2b
7,170,1901_02_TeamLG_TeamA3,TeamA3,TeamLG,NA,227,6,Bottom,3986,6025,1217,0,0,0,3,nothing,2b
8,171,1901_02_TeamLG_TeamA3,TeamA3,TeamLG,NA,228,6,Bottom,3986,6025,1217,0,0,0,3,nothing,2b
9,214,1901_02_TeamLG_TeamA3,TeamA3,TeamLG,NA,282,7,Bottom,3986,6025,2911,0,1470,0,4,other,2b


In [ ]:
df %>%
  count(type)

type,n
<chr>,<int>
alone,17
caught,20
nothing,26
other,118
stolen,70


Now we need to further look into what's going on with the "other" rows to see if these attempts are stolen/caught stealing.

In [ ]:
for (i in 1:(nrow(df) - 1)) {
  if (df[i, "type"] == "other" & df[i,"batter"] != df[i+1, "batter"] &
      df[i, "stolen_event_id"] == df[i+1, "stolen_event_id"]) {
      df[i, "type"] = "batChange"
      df[i+1, "type"] = "batChange"
}

  else if (df[i, "type"] == "other" & df[i,"batter"] == df[i+1, "batter"] &
    df[i,"first_baserunner"] == df[i+1, "first_baserunner"] &
     df[i,"second_baserunner"] == df[i+1, "second_baserunner"] &
     df[i,"third_baserunner"] == df[i+1, "third_baserunner"] &
      df[i, "stolen_event_id"] == df[i+1, "stolen_event_id"]) {
        df[i, "type"] = "noChange"
      df[i+1, "type"] = "noChange"

  }
}

In [ ]:
df %>%
  count(type)

type,n
<chr>,<int>
alone,17
batChange,74
caught,20
noChange,44
nothing,26
stolen,70


Here I did further sorting to see what was happening during the "other" plays.

In [ ]:
for (i in 1:(nrow(df) - 1)) {
  #sorting when the batter is seen on a base in the next play
  if (df[i, "type"] == "batChange" & df[i,"batter"] == df[i+1, "first_baserunner"] &
      df[i, "stolen_event_id"] == df[i+1, "stolen_event_id"]) {
      df[i, "type"] = "batterRuns"
      df[i+1, "type"] = "batterRuns"
}

  else if (df[i, "type"] == "batChange" & df[i,"batter"] == df[i+1, "second_baserunner"] &
      df[i, "stolen_event_id"] == df[i+1, "stolen_event_id"]) {
      df[i, "type"] = "batterRuns"
      df[i+1, "type"] = "batterRuns"
}

  else if (df[i, "type"] == "batChange" & df[i,"batter"] == df[i+1, "third_baserunner"] &
      df[i, "stolen_event_id"] == df[i+1, "stolen_event_id"]) {
      df[i, "type"] = "batterRuns"
      df[i+1, "type"] = "batterRuns"
}

#sorting when the sides change
else if (df[i, "type"] == "batChange" & df[i,"top_bottom_inning"] != df[i+1, "top_bottom_inning"] &
      df[i, "stolen_event_id"] == df[i+1, "stolen_event_id"]) {
      df[i, "type"] = "sideChange"
      df[i+1, "type"] = "sideChange"

      }
}

After final sorting and dicussing with teammates, we decided which cases to include in our final stolen vs caught stealing plays.

In [ ]:
df %>%
  count(type)
# include stoleThird, *stolen, stolen as overall stolen also include side change
# include caught as caught stolen
# get rid of batter runs, nothing changes, batter changes

type,n
<chr>,<int>
alone,17
batChange,12
batterRuns,24
caught,20
noChange,44
nothing,26
sideChange,38
stolen,70


In [ ]:
df

,...1,game_str,home_team,away_team,at_bat,play_per_game,inning,top_bottom_inning,pitcher,catcher,batter,first_baserunner,second_baserunner,third_baserunner,stolen_event_id,type,throw
,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
1,46,1900_05_TeamKK_TeamB,TeamB,TeamKK,NA,56,2,Bottom,7119,3304,2488,0,0,0,0,nothing,2b
2,47,1900_05_TeamKK_TeamB,TeamB,TeamKK,NA,57,2,Bottom,7119,3304,2488,0,0,0,0,nothing,2b
3,204,1900_05_TeamKK_TeamB,TeamB,TeamKK,NA,219,7,Top,1516,1041,6188,6861,4140,0,1,noChange,2b
4,205,1900_05_TeamKK_TeamB,TeamB,TeamKK,NA,220,7,Top,1516,1041,6188,6861,4140,0,1,noChange,2b
5,255,1900_08_TeamKL_TeamB,TeamB,TeamKL,NA,270,9,Top,2995,1041,6092,0,3833,0,2,noChange,2b
6,256,1900_08_TeamKL_TeamB,TeamB,TeamKL,NA,271,9,Top,2995,1041,6092,0,3833,0,2,noChange,2b
7,170,1901_02_TeamLG_TeamA3,TeamA3,TeamLG,NA,227,6,Bottom,3986,6025,1217,0,0,0,3,nothing,2b
8,171,1901_02_TeamLG_TeamA3,TeamA3,TeamLG,NA,228,6,Bottom,3986,6025,1217,0,0,0,3,nothing,2b
9,214,1901_02_TeamLG_TeamA3,TeamA3,TeamLG,NA,282,7,Bottom,3986,6025,2911,0,1470,0,4,noChange,2b


Below we finally marked which attempts were successful outs (caught stealing - 1) or which attempts were failed outs (stolen base - 0)

In [ ]:
df$success


for (i in 1:(nrow(df))) {
  if (df[i, "type"] == "*stoleThird" | df[i, "type"] == "*stolen" |
    df[i, "type"] == "stolen" | df[i, "type"] == "sideChange") {
      df[i, "success"] = 0
  }

   if (df[i, "type"] == "caught") {
      df[i, "success"] = 1
  }
}

NULL

In [ ]:
df

,...1,game_str,home_team,away_team,at_bat,play_per_game,inning,top_bottom_inning,pitcher,catcher,batter,first_baserunner,second_baserunner,third_baserunner,stolen_event_id,type,throw,success
,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
1,46,1900_05_TeamKK_TeamB,TeamB,TeamKK,NA,56,2,Bottom,7119,3304,2488,0,0,0,0,nothing,2b,NA
2,47,1900_05_TeamKK_TeamB,TeamB,TeamKK,NA,57,2,Bottom,7119,3304,2488,0,0,0,0,nothing,2b,NA
3,204,1900_05_TeamKK_TeamB,TeamB,TeamKK,NA,219,7,Top,1516,1041,6188,6861,4140,0,1,noChange,2b,NA
4,205,1900_05_TeamKK_TeamB,TeamB,TeamKK,NA,220,7,Top,1516,1041,6188,6861,4140,0,1,noChange,2b,NA
5,255,1900_08_TeamKL_TeamB,TeamB,TeamKL,NA,270,9,Top,2995,1041,6092,0,3833,0,2,noChange,2b,NA
6,256,1900_08_TeamKL_TeamB,TeamB,TeamKL,NA,271,9,Top,2995,1041,6092,0,3833,0,2,noChange,2b,NA
7,170,1901_02_TeamLG_TeamA3,TeamA3,TeamLG,NA,227,6,Bottom,3986,6025,1217,0,0,0,3,nothing,2b,NA
8,171,1901_02_TeamLG_TeamA3,TeamA3,TeamLG,NA,228,6,Bottom,3986,6025,1217,0,0,0,3,nothing,2b,NA
9,214,1901_02_TeamLG_TeamA3,TeamA3,TeamLG,NA,282,7,Bottom,3986,6025,2911,0,1470,0,4,noChange,2b,NA


Below just removes any cases where no stolen base attempt was made. base on the sorting from above

In [ ]:
df_two_plays_final =
  df %>% filter(success == 0 | success == 1)

In [ ]:
df_two_plays_final

...1,game_str,home_team,away_team,at_bat,play_per_game,inning,top_bottom_inning,pitcher,catcher,batter,first_baserunner,second_baserunner,third_baserunner,stolen_event_id,type,throw,success
<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
76,1902_08_TeamMJ_TeamB,TeamB,TeamMJ,21,80,3,Top,2764,2774,9969,8240,0,0,20,stolen,2b,0
77,1902_08_TeamMJ_TeamB,TeamB,TeamMJ,21,81,3,Top,2764,2774,9969,0,8240,0,20,stolen,2b,0
163,1902_08_TeamMJ_TeamB,TeamB,TeamMJ,43,169,5,Bottom,6527,3238,1607,1180,0,0,21,stolen,2b,0
164,1902_08_TeamMJ_TeamB,TeamB,TeamMJ,43,170,5,Bottom,6527,3238,1607,0,1180,0,21,stolen,2b,0
216,1902_08_TeamMJ_TeamB,TeamB,TeamMJ,56,222,7,Top,1330,2774,9969,9384,0,0,22,sideChange,2b,0
217,1902_08_TeamMJ_TeamB,TeamB,TeamMJ,57,223,7,Bottom,5251,3238,2774,0,0,0,22,sideChange,2b,0
286,1902_09_TeamMJ_TeamB,TeamB,TeamMJ,80,296,9,Top,1768,2925,4103,8642,0,0,23,stolen,2b,0
287,1902_09_TeamMJ_TeamB,TeamB,TeamMJ,80,297,9,Top,1768,2925,4103,0,8642,0,23,stolen,2b,0
57,1902_12_TeamMI_TeamA3,TeamA3,TeamMI,17,61,2,Bottom,9158,7257,2569,2696,0,1119,25,caught,2b,1


In [ ]:
df_final <- data.frame()
for (i in 1:nrow(df_two_plays_final)) {
  if (i %% 2 == 1) {  # Add odd-indexed rows (every other row)
    df_final <- rbind(df_final, df_two_plays_final[i, ])
  }
}
df_final

,...1,game_str,home_team,away_team,at_bat,play_per_game,inning,top_bottom_inning,pitcher,catcher,batter,first_baserunner,second_baserunner,third_baserunner,stolen_event_id,type,throw,success
,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
1,76,1902_08_TeamMJ_TeamB,TeamB,TeamMJ,21,80,3,Top,2764,2774,9969,8240,0,0,20,stolen,2b,0
3,163,1902_08_TeamMJ_TeamB,TeamB,TeamMJ,43,169,5,Bottom,6527,3238,1607,1180,0,0,21,stolen,2b,0
5,216,1902_08_TeamMJ_TeamB,TeamB,TeamMJ,56,222,7,Top,1330,2774,9969,9384,0,0,22,sideChange,2b,0
7,286,1902_09_TeamMJ_TeamB,TeamB,TeamMJ,80,296,9,Top,1768,2925,4103,8642,0,0,23,stolen,2b,0
9,57,1902_12_TeamMI_TeamA3,TeamA3,TeamMI,17,61,2,Bottom,9158,7257,2569,2696,0,1119,25,caught,2b,1
11,99,1902_23_TeamMA_TeamA1,TeamA1,TeamMA,26,99,3,Bottom,7761,7415,1771,1600,0,1297,31,sideChange,2b,0
13,34,1902_24_TeamMA_TeamA1,TeamA1,TeamMA,8,34,1,Bottom,5706,7686,1771,2973,0,0,33,stolen,2b,0
15,90,1902_31_TeamMF_TeamA2,TeamA2,TeamMF,26,101,3,Bottom,7601,9774,2207,2808,0,0,38,stolen,2b,0
17,258,1903_01_TeamNE_TeamA2,TeamA2,TeamNE,68,260,7,Bottom,4044,7071,2382,2790,0,0,40,stolen,2b,0


This is our final dataset! It includes the game string and the plays of all the stolen base attempts, and also includes whether it was successful or not.

Below simply allows us to create a csv with the above dataframe, and further I was able to connect these successes vs failures with other csv files about ball velocity, pop time, and runners that other teammates created

In [ ]:
install.packages("googledrive")
library(googledrive)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
ball_velocity = read_csv("/content/ball_velocity_with_game_str.csv")

New names:
• `` -> `...1`
Rows: 111 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): game_str, passed_to
dbl (12): ...1, game_id, play_id, timestamp, game_event_player_pos, event_co...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [ ]:
ball_velocity

...1,game_str,game_id,play_id,timestamp,game_event_player_pos,event_code,coords_player_pos,field_x,field_y,distance_ft,time_ms,velocity_m_per_s,passed_to
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,1900_05_TeamKK_TeamB,5,56,2233184,4,2,4,1.0953,125.0583,129.91690,1288,30.74431,2B
2,1900_05_TeamKK_TeamB,5,218,8287906,4,2,4,-0.2751,125.7708,127.29066,1353,28.67568,2B
3,1900_08_TeamKL_TeamB,8,270,10237306,4,2,4,61.0839,64.2894,82.60228,957,26.30844,2B
4,1901_02_TeamLG_TeamA3,11,99,4238996,6,2,6,-0.7908,128.6013,130.10727,1250,31.72536,SS
5,1901_02_TeamLG_TeamA3,11,282,10908790,4,2,4,-0.9615,126.2736,128.76220,1300,30.18978,2B
6,1901_05_TeamLI_TeamA3,14,81,2539203,6,2,6,2.4216,122.1453,124.77622,1250,30.42543,SS
7,1901_07_TeamLK_TeamB,16,122,4259800,4,2,4,-0.4470,124.5951,127.18487,1200,32.30496,2B
8,1901_09_TeamLK_TeamB,18,64,2470233,4,2,4,0.2535,125.1051,129.00867,1250,31.45748,2B
9,1901_09_TeamLK_TeamB,18,145,5583380,6,2,6,-1.3560,125.5254,127.87152,1350,28.87055,SS


In [ ]:
small_final = df_final %>%
    select(game_str, play_per_game, success)

small_final

,game_str,play_per_game,success
,<chr>,<dbl>,<dbl>
1,1902_08_TeamMJ_TeamB,80,0
3,1902_08_TeamMJ_TeamB,169,0
5,1902_08_TeamMJ_TeamB,222,0
7,1902_09_TeamMJ_TeamB,296,0
9,1902_12_TeamMI_TeamA3,61,1
11,1902_23_TeamMA_TeamA1,99,0
13,1902_24_TeamMA_TeamA1,34,0
15,1902_31_TeamMF_TeamA2,101,0
17,1903_01_TeamNE_TeamA2,260,0


In [ ]:
test = combined_event_data %>% distinct(game_str, play_per_game, play_id)
test

game_str,play_per_game,play_id
<chr>,<dbl>,<dbl>
1900_01_TeamKJ_TeamB,1,1
1900_01_TeamKJ_TeamB,2,2
1900_01_TeamKJ_TeamB,3,3
1900_01_TeamKJ_TeamB,4,4
1900_01_TeamKJ_TeamB,5,5
1900_01_TeamKJ_TeamB,6,6
1900_01_TeamKJ_TeamB,7,7
1900_01_TeamKJ_TeamB,8,8
1900_01_TeamKJ_TeamB,9,9


In [ ]:
dff <- left_join(small_final, test, by=c("game_str", "play_per_game"))
dff <- dff %>% distinct()
dff

game_str,play_per_game,success,play_id
<chr>,<dbl>,<dbl>,<dbl>
1902_08_TeamMJ_TeamB,80,0,80
1902_08_TeamMJ_TeamB,169,0,169
1902_08_TeamMJ_TeamB,222,0,222
1902_09_TeamMJ_TeamB,296,0,296
1902_12_TeamMI_TeamA3,61,1,61
1902_23_TeamMA_TeamA1,99,0,99
1902_24_TeamMA_TeamA1,34,0,34
1902_31_TeamMF_TeamA2,101,0,101
1903_01_TeamNE_TeamA2,260,0,259


In [ ]:
ball_velocity = read_csv("/content/ball_velocity_with_game_str.csv")

New names:
• `` -> `...1`
Rows: 111 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): game_str, passed_to
dbl (12): ...1, game_id, play_id, timestamp, game_event_player_pos, event_co...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [ ]:
new_ball_velo <- left_join(dff, ball_velocity, by=c("game_str", "play_id"))

In [ ]:
write.csv(new_ball_velo, file = "ball_velocity_with_success.csv")

In [ ]:
pop = read_csv("/content/pop_time_combined.csv")

New names:
• `` -> `...1`
Rows: 135 Columns: 8
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): game_str
dbl (7): ...1, game_id, play_id, timestamp, player_position, event_code, pop...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [ ]:
pop_time_with_success <- left_join(dff, pop, by=c("game_str", "play_id"))
pop_time_with_success <- pop_time_with_success %>% distinct()
pop_time_with_success

game_str,play_per_game,success,play_id,...1,game_id,timestamp,player_position,event_code,pop_time
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1902_08_TeamMJ_TeamB,80,0,80,39,35,2887744,4,2,0.650
1902_08_TeamMJ_TeamB,169,0,169,41,35,6563344,4,2,0.650
1902_08_TeamMJ_TeamB,169,0,169,42,35,6565544,6,2,0.650
1902_08_TeamMJ_TeamB,222,0,222,43,35,8463344,4,2,0.700
1902_09_TeamMJ_TeamB,296,0,296,45,36,11093955,4,2,0.800
1902_12_TeamMI_TeamA3,61,1,61,49,39,1944654,4,2,0.700
1902_23_TeamMA_TeamA1,99,0,99,65,52,3242588,4,2,0.800
1902_24_TeamMA_TeamA1,34,0,34,67,53,925643,4,2,0.693
1902_31_TeamMF_TeamA2,101,0,101,81,61,2898396,4,2,0.550


In [ ]:
write.csv(pop_time_with_success, file = "pop_time_with_success.csv") #OLD TABLE (not found in submission)

Updates combining more CSV files!

In [ ]:
updated_runner = read_csv("/content/first_base_runner_distance.csv")
updated_runner

New names:
• `` -> `...1`
Rows: 118 Columns: 8
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): game_str
dbl (7): ...1, game_id, play_id, timestamp, field_x, field_y, distance_from_...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


...1,game_str,game_id,play_id,timestamp,field_x,field_y,distance_from_1st_base_ft
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1900_05_TeamKK_TeamB,5,56,2231071,41.5797,85.6737,30.56287
2,1900_05_TeamKK_TeamB,5,218,8285596,47.3886,82.0917,24.01552
3,1900_08_TeamKL_TeamB,8,270,10234600,30.4584,99.0000,47.89283
4,1901_02_TeamLG_TeamA3,11,99,4237096,39.8235,86.6019,32.45507
5,1901_02_TeamLG_TeamA3,11,227,8791992,49.5582,80.9127,21.74014
6,1901_02_TeamLG_TeamA3,11,282,10906890,41.2704,85.1508,30.40588
7,1901_05_TeamLI_TeamA3,14,12,306255,45.3867,82.8504,25.90118
8,1901_05_TeamLI_TeamA3,14,12,306255,45.3867,82.8504,25.90118
9,1901_05_TeamLI_TeamA3,14,81,2537253,42.2970,85.5834,30.00412


In [ ]:
updated_runner_distance_with_success <- left_join(dff, updated_runner, by=c("game_str", "play_id"))
updated_runner_distance_with_success <- updated_runner_distance_with_success %>% distinct()
updated_runner_distance_with_success

game_str,play_per_game,success,play_id,...1,game_id,timestamp,field_x,field_y,distance_from_1st_base_ft
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1902_08_TeamMJ_TeamB,80,0,80,37,35,2885844,41.9469,85.3257,30.057523
1902_08_TeamMJ_TeamB,169,0,169,39,35,6559094,42.0930,83.7768,28.855079
1902_08_TeamMJ_TeamB,169,0,169,40,35,6559094,42.0930,83.7768,28.855079
1902_08_TeamMJ_TeamB,222,0,222,41,35,8461444,43.0839,85.2609,29.233752
1902_09_TeamMJ_TeamB,296,0,296,43,36,11091955,39.2952,86.8743,33.021653
1902_12_TeamMI_TeamA3,61,1,61,46,39,1942754,41.5290,86.5002,31.198463
1902_23_TeamMA_TeamA1,99,0,99,60,52,3240588,45.5721,82.8603,25.784566
1902_24_TeamMA_TeamA1,34,0,34,62,53,923762,39.2385,87.8166,33.730831
1902_31_TeamMF_TeamA2,101,0,101,72,61,2896546,35.2653,92.9892,40.216092


In [ ]:
write.csv(updated_runner_distance_with_success, file = "first_base_runner_distance_with_success.csv")

In [ ]:
small_velocity = ball_velocity %>% distinct(game_str, play_id, velocity_m_per_s)
small_velocity

game_str,play_id,velocity_m_per_s
<chr>,<dbl>,<dbl>
1900_05_TeamKK_TeamB,56,30.74431
1900_05_TeamKK_TeamB,218,28.67568
1900_08_TeamKL_TeamB,270,26.30844
1901_02_TeamLG_TeamA3,99,31.72536
1901_02_TeamLG_TeamA3,282,30.18978
1901_05_TeamLI_TeamA3,81,30.42543
1901_07_TeamLK_TeamB,122,32.30496
1901_09_TeamLK_TeamB,64,31.45748
1901_09_TeamLK_TeamB,145,28.87055


In [ ]:
pop_with_success = read_csv("/content/pop_time_with_success.csv") #OLD TABLE (not found in submission)
pop_with_success

New names:
• `` -> `...1`
• `...1` -> `...6`
Rows: 65 Columns: 11
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): game_str
dbl (10): ...1, play_per_game, success, play_id, ...6, game_id, timestamp, p...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


...1,game_str,play_per_game,success,play_id,...6,game_id,timestamp,player_position,event_code,pop_time
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1902_08_TeamMJ_TeamB,80,0,80,39,35,2887744,4,2,0.650
2,1902_08_TeamMJ_TeamB,169,0,169,41,35,6563344,4,2,0.650
3,1902_08_TeamMJ_TeamB,169,0,169,42,35,6565544,6,2,0.650
4,1902_08_TeamMJ_TeamB,222,0,222,43,35,8463344,4,2,0.700
5,1902_09_TeamMJ_TeamB,296,0,296,45,36,11093955,4,2,0.800
6,1902_12_TeamMI_TeamA3,61,1,61,49,39,1944654,4,2,0.700
7,1902_23_TeamMA_TeamA1,99,0,99,65,52,3242588,4,2,0.800
8,1902_24_TeamMA_TeamA1,34,0,34,67,53,925643,4,2,0.693
9,1902_31_TeamMF_TeamA2,101,0,101,81,61,2898396,4,2,0.550


In [ ]:
updated_pop_with_success <- left_join(pop_with_success, small_velocity, by=c("game_str", "play_id"))
updated_pop_with_success <- updated_pop_with_success %>% distinct()
updated_pop_with_success

...1,game_str,play_per_game,success,play_id,...6,game_id,timestamp,player_position,event_code,pop_time,velocity_m_per_s
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1902_08_TeamMJ_TeamB,80,0,80,39,35,2887744,4,2,0.650,31.59239
2,1902_08_TeamMJ_TeamB,169,0,169,41,35,6563344,4,2,0.650,NA
3,1902_08_TeamMJ_TeamB,169,0,169,42,35,6565544,6,2,0.650,NA
4,1902_08_TeamMJ_TeamB,222,0,222,43,35,8463344,4,2,0.700,31.48230
5,1902_09_TeamMJ_TeamB,296,0,296,45,36,11093955,4,2,0.800,32.12095
6,1902_12_TeamMI_TeamA3,61,1,61,49,39,1944654,4,2,0.700,32.19064
7,1902_23_TeamMA_TeamA1,99,0,99,65,52,3242588,4,2,0.800,32.67431
8,1902_24_TeamMA_TeamA1,34,0,34,67,53,925643,4,2,0.693,32.52840
9,1902_31_TeamMF_TeamA2,101,0,101,81,61,2898396,4,2,0.550,29.17044


In [ ]:
write.csv(updated_pop_with_success, file = "pop_time_with_success_and_velocity.csv")